In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

!pip install ipywidgets

In [1]:
from huggingface_hub import notebook_login

notebook_login()

* Define a **TrainingArguments** class
  * hyperparameters, directories etc

In [ ]:
from transformers import TrainingArguments

#training_args = TrainingArguments("test-trainer")
training_args

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

[2023-12-17 17:28:59,695] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


* A new classifier head is added, randomly iinitialized

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned", save_strategy="epoch", push_to_hub=True
)

In [6]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args, 
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer)

/home/cybertron/anaconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/soonchang/bert-finetuned into local empty directory.


In [7]:
trainer.train()

/home/cybertron/anaconda3/envs/llava/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: soonchangpoh. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/cybertron/anaconda3/envs/llava/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.381200


/home/cybertron/anaconda3/envs/llava/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/cybertron/anaconda3/envs/llava/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=690, training_loss=0.30923866050830784, metrics={'train_runtime': 175.821, 'train_samples_per_second': 62.586, 'train_steps_per_second': 3.924, 'total_flos': 430291408824720.0, 'train_loss': 0.30923866050830784, 'epoch': 3.0})

In [9]:
from transformers import AutoModelForMaskedLM , AutoTokenizer

checkpoint = "camembert-base"
model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

In [10]:
model.push_to_hub("dummy-model")
tokenizer.push_to_hub("dummy-model")

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/soonchang/dummy-model/commit/92b8293e6b9f901134abe30c7dca0f37a8e81114', commit_message='Upload tokenizer', commit_description='', oid='92b8293e6b9f901134abe30c7dca0f37a8e81114', pr_url=None, pr_revision=None, pr_num=None)

**compute_metric()**
* Input: EvalPrediction (tuple with **predictions** & **label_ids**)
* Return: Dictionary with key (name of metric) and val (float value)

In [ ]:
predictions = trainer.predict(tokenized_datasets['validation'])
print(predictions.predictions.shape)
print(predictions.label_ids.shape)

In [ ]:
predictions

In [ ]:
# transform predictions

import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
!pip install evaluate
!pip install sklearn scipy

In [ ]:
import evaluate
metric = evaluate.load("glue","mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue","mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer",evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [18]:
from huggingface_hub import (
    # User management
    login,
    logout,
    whoami,

    # Repository creation and management
    create_repo,
    delete_repo,
    update_repo_visibility,

    # And some methods to retrieve/change information about the content
    list_models,
    list_datasets,
    list_metrics,
    list_repo_files,
    upload_file,
    delete_file,
)

In [19]:
from huggingface_hub import create_repo

create_repo("dummy-test")

RepoUrl('https://huggingface.co/soonchang/dummy-test', endpoint='https://huggingface.co', repo_type='model', repo_id='soonchang/dummy-test')

Upload file

* Method 1 **upload_file**
   * doesnt require git & git-lfs
   * Push using HTTP POST requests
   * For <5GB files

In [22]:
from huggingface_hub import upload_file
path = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/huggingface_nlp_course/notebooks/bert-finetuned/config.json'
upload_file(
    path ,
    path_in_repo="config.json",
    repo_id="soonchang/dummy-test",
)

TypeError: HfApi.upload_file() takes 1 positional argument but 2 positional arguments (and 2 keyword-only arguments) were given

## Repository class
* git like

In [23]:
from huggingface_hub import Repository

repo = Repository("dummy-test", clone_from="soonchang/dummy-test")
# create dummy-test in local working directory


/home/cybertron/anaconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/soonchang/dummy-test into local empty directory.


In [24]:
# make sure local clone is up to date
repo.git_pull()

In [25]:
model.save_pretrained("dummy_test")
tokenizer.save_pretrained("dummy_test")

('dummy_test/tokenizer_config.json',
 'dummy_test/special_tokens_map.json',
 'dummy_test/sentencepiece.bpe.model',
 'dummy_test/added_tokens.json',
 'dummy_test/tokenizer.json')

In [26]:
repo.git_add()
repo.git_commit("Add model & tokenizer files")
repo.git_push()

Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.87k [00:00<?, ?B/s]

To https://huggingface.co/soonchang/dummy-test
   55bc536..de5c8e5  main -> main



'https://huggingface.co/soonchang/dummy-test/commit/de5c8e550e35ee0898f211e7cf15dffd207a0475'

* git-based approach